* sklearnデータセットに収録されたカリフォルニアの住宅価格のデータセットで回帰モデルを学習

In [ ]:
# 回帰モデル学習用のライブラリのインポート
import pandas as pd
import openpyxl
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import math

* データの内容 カルフォルニアの国勢調査ブロックごとの統計情報と住宅価格の中央値をデータ化
  * https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html
  * https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset
  * 8の説明変数と応答変数
* feature data
  * MedInc : 対象ブロックの年収の中央値
  * HouseAge : 建物築年数の中央値
  * AveRooms : 平均部屋数
  * AveBedrms : 寝室数の平均
  * Population : 対象ブロックの人口
  * AveOccup : 一軒に住む人数の平均
  * Latitude : 緯度
  * Longitude : 経度
* target data
  * カリフォルニア地区の国勢調査ブロックごとの住宅価格の中央値

In [ ]:
# sklearnデータセットに収録されたカルフォルニアの住宅価格のデータセットをロード
from sklearn.datasets import fetch_california_housing
housing_data = fetch_california_housing()  # インターネットにアクセスしてダウンロード

# 学習用データのデータフレームを作成
x = pd.DataFrame(housing_data["data"],columns=housing_data["feature_names"])
y = pd.DataFrame(housing_data["target"],columns=["target"])

# データフレームの形状と説明変数の列名を表示
print('feature shape:', x.shape, housing_data["feature_names"])
print('target shape:', y.shape)

In [ ]:
## 欠損値の確認 (pandasのisnull関数を利用)
# any(axis=0) で各列に欠損値があるかを確認
print(x.isnull().any(axis=0))
print(y.isnull().any(axis=0))

In [ ]:
# データを学習用と検証用に分割 (test_sizeで検証データの比率を指定(0.0-1.0の間), random_stateは乱数シード)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=1)
print(len(x_train), len(x_test), len(y_train), len(y_test)) # それぞれのデータ数を確認

In [ ]:
# 学習データ(説明変数と応答変数)の相関行列を確認
train = pd.concat([x_train, y_train],axis=1,sort=False)
train.corr()

In [ ]:
import seaborn as sns
# 学習用データのペアプロットを表示
sns.pairplot(train)
plt.show()

In [ ]:
# 学習用データのヒートマップを表示
plt.rcParams['figure.figsize'] = (15.0, 15.0)
sns.heatmap(train.corr(), square=True, annot=True, cmap='coolwarm')
plt.show()

In [ ]:
# 独立変数間の分散拡大要因を確認 (データサイエンス基礎 配布資料(第8回) 参照)
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
import statsmodels.api as sm

sm_model = sm.OLS(y_train, x_train)
result = sm_model.fit()
# print (result.summary())
num_cols = sm_model.exog.shape[1] # 説明変数の列数
vifs = [vif(sm_model.exog, i) for i in range(0, num_cols)]
print(pd.DataFrame(vifs, index=sm_model.exog_names, columns=['VIF']))

In [ ]:
# 回帰モデルを作成
model = LinearRegression()

# 回帰モデルを学習用データで学習
model.fit(x_train, y_train)

## 検証用データを利用してモデルを評価
# 学習したモデルを使い、検証用データから応答変数を予測
pred = model.predict(x_test)

# 平均絶対誤差、平均二乗誤差、決定係数を計算
print('平均絶対誤差', mean_absolute_error(y_pred=pred, y_true=y_test))
print('平均二乗誤差', mean_squared_error(y_pred=pred, y_true=y_test))
print('平均平方二乗誤差', math.sqrt(mean_squared_error(y_pred=pred, y_true=y_test)))
print('決定係数', model.score(x_test, y_test))
print('(参考) 学習データに対する決定係数', model.score(x_train, y_train))

In [ ]:
# sklearn の RFECV を使って変数を選択する
# RFECV : クロスバリデーションを行いながら説明変数の除外を実施
from sklearn.feature_selection import RFECV
import sklearn
rfecv = RFECV(estimator=LinearRegression(), cv=10, scoring="r2")
rfecv.fit(x_train.values, y_train)

# 選択された説明変数
print('all: ', x_train.columns.values)
print('selected:', x_train.columns[rfecv.support_].values)

# 選択された説明変数のデータフレームを作成
x_train_mod = x_train[x_train.columns[rfecv.support_]]
x_test_mod = x_test[x_test.columns[rfecv.support_]]

In [ ]:
# 修正版の回帰モデルを作成
model_mod = LinearRegression()

# 修正版の回帰モデルを変数選択後の学習用データで学習
model_mod.fit(x_train_mod, y_train)

## 検証用データを利用してモデルを評価
# 学習したモデルを使い、検証用データから応答変数を予測
pred = model_mod.predict(x_test_mod)

# 平均絶対誤差、平均二乗誤差、決定係数を計算
print('平均絶対誤差', mean_absolute_error(y_pred=pred, y_true=y_test))
print('平均二乗誤差', mean_squared_error(y_pred=pred, y_true=y_test))
print('平均平方二乗誤差', math.sqrt(mean_squared_error(y_pred=pred, y_true=y_test)))
print('決定係数', model_mod.score(x_test_mod, y_test))
print('(参考) 学習データに対する決定係数', model_mod.score(x_train_mod, y_train))

In [ ]:
sm_model = sm.OLS(y_train, x_train_mod)
result = sm_model.fit()
# print (result.summary())
num_cols = sm_model.exog.shape[1] # 説明変数の列数
vifs = [vif(sm_model.exog, i) for i in range(0, num_cols)]
print(pd.DataFrame(vifs, index=sm_model.exog_names, columns=['VIF']))